In [1]:
import cv2
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.cluster.vq import kmeans,vq
from sklearn.svm import LinearSVC

# Define training path

In [2]:
train_path = 'train'
train_dirs = os.listdir(train_path)

image_list = []
image_classes_list = []

for idx, image_dir in enumerate(train_dirs):
    image_name_path = os.path.join(train_path, image_dir)
    for image_name in os.listdir(image_name_path):
        image_path = os.path.join(image_name_path, image_name)
        image_list.append(image_path)

        image_classes_list.append(idx)

# Find features / corner

In [3]:
sift = cv2.SIFT_create()

descriptor_list = []
for image in image_list:
    _,desc = sift.detectAndCompute(cv2.imread(image),None)
    descriptor_list.append(desc)

In [4]:
print('shape = ', descriptor_list[0].shape)
print(descriptor_list)

shape =  (103, 128)
[array([[ 2.,  1.,  0., ...,  1.,  1.,  2.],
       [ 2.,  1.,  0., ...,  0.,  1.,  6.],
       [ 2.,  2.,  1., ...,  0.,  0.,  0.],
       ...,
       [75.,  5.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  4.],
       [72., 38.,  0., ...,  3.,  5.,  1.]], dtype=float32), array([[ 1.,  0.,  0., ...,  7.,  6., 96.],
       [21.,  3.,  0., ...,  4., 28., 78.],
       [21.,  6.,  0., ..., 69.,  7.,  1.],
       ...,
       [ 4., 15., 12., ...,  6.,  4., 30.],
       [14.,  2.,  0., ..., 17., 18., 19.],
       [ 0.,  0.,  0., ..., 26.,  9., 14.]], dtype=float32), array([[ 1.,  1., 15., ...,  9.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  5.],
       [ 2.,  0.,  0., ...,  0.,  0.,  1.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  2.],
       [ 1.,  0.,  0., ...,  1.,  1.,  2.]], dtype=float32), array([[ 2.,  5.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 53.,  3., 11.],
       [10.,

# Change dimensionality by Vstack

In [5]:
stacked_descriptor_list = descriptor_list[0]
for desc in descriptor_list[1:]:
    stacked_descriptor_list = np.vstack((stacked_descriptor_list,desc)) #stack descriptor to 2d

In [6]:
print('shape = ',stacked_descriptor_list.shape)
print(stacked_descriptor_list)

shape =  (11799, 128)
[[ 2.  1.  0. ...  1.  1.  2.]
 [ 2.  1.  0. ...  0.  1.  6.]
 [ 2.  2.  1. ...  0.  0.  0.]
 ...
 [ 3.  5.  0. ... 66. 52. 12.]
 [ 0.  0.  0. ...  5.  5. 21.]
 [ 1.  0.  0. ... 15. 17.  1.]]


# Find kmeans / cluster for all descriptor (bag of word)

In [7]:
centroids, _ = kmeans(stacked_descriptor_list, 100, 1) #value, total cluster, epoch

print(centroids)
print(_)

image_features = np.zeros((len(image_list), len(centroids)), "float32")

for i in range(len(image_list)):
    #vq used to find which centroid closer with the descriptor
    words, _ = vq(descriptor_list[i], centroids)  
    for word in words:
        image_features[i][word] += 1

[[14.12605   11.411765   7.319328  ... 11.235294  15.840336  19.403362 ]
 [77.03615   40.21687   15.421687  ...  5.518072   3.2650602 11.253012 ]
 [ 6.9886365  5.9772725 13.431818  ...  5.6477275  6.943182   9.590909 ]
 ...
 [31.401709  16.65812   16.316238  ...  5.3675213  5.008547  17.017094 ]
 [ 8.6470585 14.658824  31.258823  ... 22.517647  45.77647   30.882353 ]
 [16.119404   9.11194   15.4925375 ...  3.1567163  4.8208957  9.164179 ]]
274.48944


In [8]:
print(image_features)

[[1. 2. 2. ... 1. 0. 4.]
 [2. 0. 1. ... 2. 1. 1.]
 [0. 2. 0. ... 0. 0. 0.]
 ...
 [0. 3. 0. ... 3. 2. 1.]
 [3. 6. 1. ... 6. 1. 1.]
 [3. 2. 5. ... 3. 3. 5.]]


# Preprocess value

In [9]:
scaler = StandardScaler()
image_features = scaler.fit_transform(image_features)

# Classification by SVM

In [10]:
cls = LinearSVC()
cls.fit(image_features, np.array(image_classes_list))

LinearSVC()

# Testing

In [11]:
test_path = 'test'
test_dirs = os.listdir(test_path)

image_list = []

for image_name in os.listdir(test_path):
    image_path = os.path.join(test_path, image_name)
    image_list.append(image_path)

sift = cv2.SIFT_create()

descriptor_list = []
for image in image_list:
    _,desc = sift.detectAndCompute(cv2.imread(image),None)
    descriptor_list.append(desc)
    
stacked_descriptor_list = descriptor_list[0]
for desc in descriptor_list[1:]:
    stacked_descriptor_list = np.vstack((stacked_descriptor_list,desc)) #stack descriptor to 2d 
    
centroids, _ = kmeans(stacked_descriptor_list, 100, 1) #value, total cluster, epoch

image_features = np.zeros((len(image_list), len(centroids)), "float32")

for i in range(len(image_list)):
    #vq used to find which centroid closer with the descriptor
    words, _ = vq(descriptor_list[i], centroids)  
    for word in words:
        image_features[i][word] += 1
        
image_features = scaler.fit_transform(image_features)
result = cls.predict(image_features)

for image_path, class_id in zip(image_list, result):
    print('image: {} | class : {}'.format(image_path, train_dirs[class_id]))

image: test/1.jpg | class : aeroplane
image: test/2.jpg | class : aeroplane
image: test/3.jpg | class : aeroplane
image: test/4.jpg | class : bicycle
image: test/5.jpg | class : car
image: test/6.jpg | class : aeroplane
image: test/7.jpg | class : car
image: test/8.jpg | class : bicycle
image: test/9.jpg | class : car
